# NLP Code Along

For this code along we will build a spam filter! We'll use the various NLP tools we learned about as well as a new classifier, Naive Bayes.

We'll use a classic dataset for this - UCI Repository SMS Spam Detection: https://archive.ics.uci.edu/ml/datasets/SMS+Spam+Collection

In [2]:
import findspark
findspark.init()
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession.builder.appName('nlp').getOrCreate()

In [4]:
data = spark.read.csv("smsspamcollection/SMSSpamCollection",inferSchema=True,sep='\t')

In [5]:
data.show(truncate= False)

+----+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|_c0 |_c1                                                                                                                                                                                                 |
+----+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|ham |Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...                                                                                     |
|ham |Ok lar... Joking wif u oni...                                                                                                                                                     

In [6]:
data = data.withColumnRenamed( '_c0', 'class').withColumnRenamed('_c1', 'text')

In [7]:
data.show()


+-----+--------------------+
|class|                text|
+-----+--------------------+
|  ham|Go until jurong p...|
|  ham|Ok lar... Joking ...|
| spam|Free entry in 2 a...|
|  ham|U dun say so earl...|
|  ham|Nah I don't think...|
| spam|FreeMsg Hey there...|
|  ham|Even my brother i...|
|  ham|As per your reque...|
| spam|WINNER!! As a val...|
| spam|Had your mobile 1...|
|  ham|I'm gonna be home...|
| spam|SIX chances to wi...|
| spam|URGENT! You have ...|
|  ham|I've been searchi...|
|  ham|I HAVE A DATE ON ...|
| spam|XXXMobileMovieClu...|
|  ham|Oh k...i'm watchi...|
|  ham|Eh u remember how...|
|  ham|Fine if thats th...|
| spam|England v Macedon...|
+-----+--------------------+
only showing top 20 rows



In [9]:
from pyspark.sql.functions import length

In [10]:
data = data.withColumn('length', length(data['text']))

In [11]:
data.show()

+-----+--------------------+------+
|class|                text|length|
+-----+--------------------+------+
|  ham|Go until jurong p...|   111|
|  ham|Ok lar... Joking ...|    29|
| spam|Free entry in 2 a...|   155|
|  ham|U dun say so earl...|    49|
|  ham|Nah I don't think...|    61|
| spam|FreeMsg Hey there...|   147|
|  ham|Even my brother i...|    77|
|  ham|As per your reque...|   160|
| spam|WINNER!! As a val...|   157|
| spam|Had your mobile 1...|   154|
|  ham|I'm gonna be home...|   109|
| spam|SIX chances to wi...|   136|
| spam|URGENT! You have ...|   155|
|  ham|I've been searchi...|   196|
|  ham|I HAVE A DATE ON ...|    35|
| spam|XXXMobileMovieClu...|   149|
|  ham|Oh k...i'm watchi...|    26|
|  ham|Eh u remember how...|    81|
|  ham|Fine if thats th...|    56|
| spam|England v Macedon...|   155|
+-----+--------------------+------+
only showing top 20 rows



In [12]:
data.groupBy('class').mean().show()

+-----+-----------------+
|class|      avg(length)|
+-----+-----------------+
|  ham|71.45431945307645|
| spam|138.6706827309237|
+-----+-----------------+



In [13]:
from pyspark.ml.feature import Tokenizer, StopWordsRemover, CountVectorizer, IDF, StringIndexer

In [14]:
tokenizer = Tokenizer(inputCol='text', outputCol= 'token_text')

In [15]:
stop_remove = StopWordsRemover(inputCol='token_text', outputCol='stop_token')

In [16]:
count_vec = CountVectorizer(inputCol= 'stop_token', outputCol= 'c_vec')

In [17]:
idf = IDF(inputCol='c_vec', outputCol='tf_idf')

In [18]:
ham_spam_to_numeric = StringIndexer(inputCol='class', outputCol='label')

In [19]:
from pyspark.ml.feature import VectorAssembler

In [20]:
clean_up = VectorAssembler(inputCols=['tf_idf', 'length'], outputCol='features')

In [21]:
from pyspark.ml.classification import NaiveBayes

In [22]:
nb = NaiveBayes()

In [23]:
from pyspark.ml import Pipeline

In [24]:
data_prep_pipe = Pipeline(stages=[ham_spam_to_numeric, tokenizer,stop_remove,count_vec, idf, clean_up])

In [25]:
cleaner = data_prep_pipe.fit(data)

In [26]:
clean_data = cleaner.transform(data)

In [28]:
clean_data = clean_data.select('label', 'features')

In [29]:
clean_data.show()

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  0.0|(13424,[7,11,31,6...|
|  0.0|(13424,[0,24,297,...|
|  1.0|(13424,[2,13,19,3...|
|  0.0|(13424,[0,70,80,1...|
|  0.0|(13424,[36,134,31...|
|  1.0|(13424,[10,60,139...|
|  0.0|(13424,[10,53,103...|
|  0.0|(13424,[125,184,4...|
|  1.0|(13424,[1,47,118,...|
|  1.0|(13424,[0,1,13,27...|
|  0.0|(13424,[18,43,120...|
|  1.0|(13424,[8,17,37,8...|
|  1.0|(13424,[13,30,47,...|
|  0.0|(13424,[39,96,217...|
|  0.0|(13424,[552,1697,...|
|  1.0|(13424,[30,109,11...|
|  0.0|(13424,[82,214,47...|
|  0.0|(13424,[0,2,49,13...|
|  0.0|(13424,[0,74,105,...|
|  1.0|(13424,[4,30,33,5...|
+-----+--------------------+
only showing top 20 rows



In [41]:
(training,testing) = clean_data.randomSplit([0.7,0.3])

In [42]:
spam_predictor = nb.fit(training)

In [43]:
data.printSchema()

root
 |-- class: string (nullable = true)
 |-- text: string (nullable = true)
 |-- length: integer (nullable = true)



In [44]:
test_results = spam_predictor.transform(testing)

In [45]:
test_results.show()

+-----+--------------------+--------------------+--------------------+----------+
|label|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
|  0.0|(13424,[0,1,4,50,...|[-822.58878578304...|[1.0,5.7186054346...|       0.0|
|  0.0|(13424,[0,1,5,15,...|[-1000.7552374258...|[1.0,5.7197380833...|       0.0|
|  0.0|(13424,[0,1,7,8,1...|[-877.64757212853...|[1.0,6.0138224497...|       0.0|
|  0.0|(13424,[0,1,9,14,...|[-541.29794886434...|[1.0,1.1494057032...|       0.0|
|  0.0|(13424,[0,1,12,33...|[-451.92217597347...|[1.0,1.3083401358...|       0.0|
|  0.0|(13424,[0,1,14,18...|[-1381.5477188264...|[1.0,6.6342234727...|       0.0|
|  0.0|(13424,[0,1,14,78...|[-685.23640351889...|[1.0,4.4984172717...|       0.0|
|  0.0|(13424,[0,1,15,20...|[-680.20205589077...|[1.0,6.0874154231...|       0.0|
|  0.0|(13424,[0,1,18,20...|[-835.02549048956...|[1.0,4.8556903615...|       0.0|
|  0.0|(13424,[0

In [46]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [47]:
acc_eval = MulticlassClassificationEvaluator()
acc = acc_eval.evaluate(test_results)
print("Accuracy of model at predicting spam was: {}".format(acc))

Accuracy of model at predicting spam was: 0.9261088517749948
